1. Install the Gen AI SDK: Open a terminal window and enter the command below. You can also [install it in a virtualenv](https://googleapis.dev/python/aiplatform/latest/index.html)

In [ ]:
!pip install --upgrade google-genai
!export GOOGLE_CLOUD_API_KEY="YOUR_API_KEY"

2. Use the following code in your application to request a model response

In [ ]:
from google import genai
from google.genai import types
import base64
import os

def generate():
  client = genai.Client(
      vertexai=True,
      api_key=os.environ.get("GOOGLE_CLOUD_API_KEY"),
  )


  model = "gemini-2.5-flash"
  contents = [
    types.Content(
      role="user",
      parts=[
      ]
    )
  ]
  tools = [
    types.Tool(
      retrieval=types.Retrieval(
        vertex_rag_store=types.VertexRagStore(
          rag_resources=[
            types.VertexRagStoreRagResource(
              rag_corpus="projects/qwiklabs-gcp-04-0dc8cd335953/locations/us-west1/ragCorpora/7991637538768945152"
            )
          ],
        )
      )
    )
  ]

  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 65535,
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    tools = tools,
    thinking_config=types.ThinkingConfig(
      thinking_level="HIGH",
    ),
  )

  for chunk in client.models.generate_content_stream(
    model = model,
    contents = contents,
    config = generate_content_config,
    ):
    if not chunk.candidates or not chunk.candidates[0].content or not chunk.candidates[0].content.parts:
        continue
    print(chunk.text, end="")

generate()

ClientError: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Unable to submit request because Model input cannot be empty. Learn more: https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/gemini', 'status': 'INVALID_ARGUMENT'}}

In [ ]:
from vertexai.generative_models import GenerativeModel

model = GenerativeModel("gemini-2.5-flash")

DOCUMENTS = """
1. Snow removal begins when snowfall exceeds 2 inches.
   Priority: emergency routes, hospitals, highways, residential streets.

2. Emergency protocol: 12-hour plow shifts, central command center.

3. School closures: >6 inches overnight or unsafe roads by 5 AM.

4. Plow routes: 650,000 sq miles, primary cleared in 4 hours, secondary in 12.

5. FAQ: road status online, call center 6 AM–8 PM.
"""

def rag_answer(question):
    prompt = f"""
    You are an assistant for the Alaska Department of Snow.
    Use ONLY the following documents to answer:

    {DOCUMENTS}

    Question: {question}
    """
    response = model.generate_content(prompt)
    return response.text

/usr/local/lib/python3.12/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [ ]:
rag_answer("What are the snow removal priorities")

'The snow removal priorities are: emergency routes, hospitals, highways, residential streets.'